In [2]:
import os
import sys
import json
from pathlib import Path

import torch
import numpy as np
import logging
#logging.basicConfig(level=logging.DEBUG)

from einops import repeat
import seaborn as sns
import matplotlib.pyplot as plt

#from shrp.models.def_net import NNmodule
#from shrp.models.def_AE_module import AEModule
from shrp.datasets.dataset_tokens import DatasetTokens
from shrp.git_re_basin.git_re_basin import resnet18_permutation_spec
from shrp.datasets.def_FastTensorDataLoader import FastTensorDataLoader

from shrp.git_re_basin.git_re_basin import (
    resnet18_permutation_spec,
    zoo_cnn_large_permutation_spec,
    zoo_cnn_permutation_spec)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{device=}')

device='cpu'


In [3]:
# select clean and trig zoos
size = 'small' # 'small' or 'large' CNN
path_clean = Path('/ds2/model_zoos/zoos_v2/MNIST/tune_zoo_mnist_uniform/')
path_trig = Path('/ds2/model_zoos/zoos_backdoors/MNIST/tune_zoo_mnist_uniform_nc_1_p_25/')

# select hyperrep for embedding
path_hyper = Path('/netscratch2/kschuerholt/code/shrp/experiments/03_trojai/02_mnist/tune/trojai_mnist_clean_v1/AE_trainable_57787_00003_3_ae_d_model=256,ae_lat_dim=32,training_view_2_canon=False_2023-06-22_22-20-41')
hyper_chkpt = 100 # checkpoit of AE to load

# select location to save data to (None skips this)
path_eval_record = None # NOT YET IMPLEMENTED

In [4]:
# configure new datasets zoo
if size == 'small':
    permutation_spec = zoo_cnn_permutation_spec() # small CNN
else:
    permutation_spec = zoo_cnn_large_permutation_spec() # large CNN

## clean specific
result_key_list_clean = ["test_acc", "training_iteration"]
config_key_list = []
property_keys_clean = {
    "result_keys": result_key_list_clean,
    "config_keys": config_key_list,
}

## trig specific
result_key_list_trig = ["triggered/test_acc", "clean/test_acc", "training_iteration"]
property_keys_trig = {
    "result_keys": result_key_list_trig,
    "config_keys": config_key_list,
}
tokensize = 0 # discover tokensize from dataset 

In [5]:
# config to set how datasets should be downloaded. set in single place to ensure 
dataset_config = {
    'epoch_lst' : 50,
    'mode' : "vector",
    'permutation_spec' : permutation_spec,
    'map_to_canonical' : False,
    'standardize' : "l2_ind",
    'ds_split' : [0.7, 0.15, 0.15],  
    'max_samples' : 1000,
    'weight_threshold' : 15000,
    'precision' : "32",
    'filter_function' : None,  # gets sample path as argument and returns True if model needs to be filtered out
    'num_threads' : 8,
    'shuffle_path' : True,
    'verbosity' : 3,
    'getitem' : "tokens+props",
    'ignore_bn' : False,
    'tokensize' : tokensize
}

In [ ]:
# load triggered zoo -- train, test, val
trainset_clean = DatasetTokens(root=[path_clean.absolute()], train_val_test='train', property_keys=property_keys_clean, **dataset_config)
testset_clean = DatasetTokens(root=[path_clean.absolute()], train_val_test='test',  property_keys=property_keys_clean, **dataset_config)
valset_clean = DatasetTokens(root=[path_clean.absolute()], train_val_test='val',  property_keys=property_keys_clean, **dataset_config)

2023-08-16 17:14:51,670	INFO worker.py:1553 -- Started a local Ray instance.


In [48]:
valset_clean[0]

(tensor([[-0.0345, -0.0816, -0.0300,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0266, -0.0376, -0.0218,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0012,  0.0384, -0.0352,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.0788,  0.0630,  0.0752,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0399,  0.0349, -0.0091,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0515,  0.0057,  0.0345,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([[ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False]]),
 tensor([[ 0,  0,  0],
         [ 1,  0,  1],
         [ 2,  0,  2],
         [ 3,  0,  3],
         [ 4,  0,  4],
         [ 5,  0,  5],
         [ 6,  0,  6],
         [ 7,  0,  7],
    

In [23]:
# load triggered zoo -- train, test, val
trainset_trig = DatasetTokens(root=[path_trig.absolute()], train_val_test='train', property_keys=property_keys_trig, **dataset_config)
testset_trig = DatasetTokens(root=[path_trig.absolute()], train_val_test='test',  property_keys=property_keys_trig, **dataset_config)
valset_trig = DatasetTokens(root=[path_trig.absolute()], train_val_test='val',  property_keys=property_keys_trig, **dataset_config)

2023-08-16 12:09:16,317	INFO worker.py:1636 -- Started a local Ray instance.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:02<00:00, 33.03it/s]
70it [00:00, 464.89it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 1293.40it/s]
2023-08-16 12:09:28,160	INFO worker.py:1636 -- Started a local Ray instance.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.96it/s]
15it [00:00, 472.95it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 1322.23it/s]
2023-08-16 12:09:39,459	INFO worker.py:1636 -- Started a local Ray instance.
100%|██████████████████████████████████████████

In [25]:
config_hyper = json.load(path_hyper.joinpath('params.json').open('r'))
config_hyper['device']=device
module = AEModule(config_hyper).to(device)
checkpoint = torch.load(path_hyper.joinpath(f'checkpoint_{str(hyper_chkpt).zfill(6)}/state.pt'), map_location=device)

# fixing model checkpoint keys if necessary
if '_orig_mod.' in next(iter(checkpoint['model'].keys())):
    checkpoint['model'] = { k[k.find('.')+1:] : v for k, v in checkpoint['model'].items()}
    
module.model.load_state_dict(checkpoint['model'])

Global seed set to 32


model: use simclr NT_Xent loss
Running single-gpu. send model to device: cpu
++++++ USE AUTOMATIC MIXED PRECISION +++++++


<All keys matched successfully>

In [26]:
def map_embeds(dataset, model, device, dataset_config=dataset_config):
    w_ds, _   = dataset.__get_weights__()
    pos_ds = torch.stack(dataset.pos) if dataset_config['mode'] == 'tokenize' else repeat(dataset.positions, "n d -> b n d", b=w_ds.shape[0])
    z_ds   = model.forward_encoder(w_ds.to(device),pos_ds.to(device))
    return w_ds, pos_ds, z_ds

In [27]:
w_train_clean, pos_train_clean, z_train_clean = map_embeds(trainset_clean, module, device)
w_test_clean, pos_test_clean, z_test_clean = map_embeds(testset_clean, module, device)
w_val_clean, pos_val_clean, z_val_clean = map_embeds(valset_clean, module, device)

w_train_trig, pos_train_trig, z_train_trig = map_embeds(trainset_trig, module, device)
w_test_trig, pos_test_trig, z_test_trig = map_embeds(testset_trig, module, device)
w_val_trig, pos_val_trig, z_val_trig = map_embeds(valset_trig, module, device)

In [45]:
pos_train_clean[0].shape

torch.Size([48, 3])

In [28]:
# generate labels: poisoned, not-poisoend
fraction = 1.0
idx_end_train = int(z_train_trig.shape[0]*fraction)
idx_end_test = int(z_test_trig.shape[0]*fraction)
idx_end_val = int(z_val_trig.shape[0]*fraction)

z_tr = torch.cat([z_train_clean.flatten(start_dim=1),z_train_trig[:idx_end_train].flatten(start_dim=1)])
print(z_tr.shape)
z_te = torch.cat([z_test_clean.flatten(start_dim=1),z_test_trig[:idx_end_test].flatten(start_dim=1)])
print(z_te.shape)
z_vl = torch.cat([z_val_clean.flatten(start_dim=1),z_val_trig[:idx_end_val].flatten(start_dim=1)])
print(z_vl.shape)

# generate labels: poisoned, not-poisoend
w_tr = torch.cat([w_train_clean.flatten(start_dim=1),w_train_trig[:idx_end_train].flatten(start_dim=1)])
print(w_tr.shape)
w_te = torch.cat([w_test_clean.flatten(start_dim=1),w_test_trig[:idx_end_test].flatten(start_dim=1)])
print(w_te.shape)
w_vl = torch.cat([w_val_clean.flatten(start_dim=1),w_val_trig[:idx_end_val].flatten(start_dim=1)])
print(w_vl.shape)


# concat
p_tr = torch.cat([torch.zeros(z_train_clean.shape[0]),torch.ones(z_train_trig[:idx_end_train].shape[0])])
print(p_tr.shape)
p_te = torch.cat([torch.zeros(z_test_clean.shape[0]),torch.ones(z_test_trig[:idx_end_test].shape[0])])
print(p_te.shape)
p_vl = torch.cat([torch.zeros(z_val_clean.shape[0]),torch.ones(z_val_trig[:idx_end_val].shape[0])])
print(p_vl.shape)

torch.Size([770, 1536])
torch.Size([165, 1536])
torch.Size([165, 1536])
torch.Size([770, 9648])
torch.Size([165, 9648])
torch.Size([165, 9648])
torch.Size([770])
torch.Size([165])
torch.Size([165])


In [29]:
p_tr

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
def predict_poisoned(z_train, z_val, z_test, prop_train, prop_test, prop_val, verbose=True):
    ## get list of classes and class labels
    # cast to float and push to cpu. back to gpu will be done at dataloader time
    z_train = z_train.float().to(torch.device("cpu"))
    z_test = z_test.float().to(torch.device("cpu"))
    if z_val:
        z_val = z_val.float().to(torch.device("cpu"))
        
    # train set classes
    classes = list(np.unique(prop_train))
    no_classes = len(classes)
    # assert all classes are in trainset
    classes_test = list(np.unique(prop_test))
    assert set(classes_test).issubset(set(classes)), "test set contains classes which are not in train set"
    if prop_val:
        classes_val = list(np.unique(prop_val))
        assert set(classes_val).issubset(set(classes)), "val set contains classes which are not in train set"

    # one hot encoding
    labels_train = torch.tensor([float(classes.index(vdx)) for idx, vdx in enumerate(prop_train)]).long()
    labels_test = torch.tensor([float(classes.index(vdx)) for idx, vdx in enumerate(prop_test)]).long()
    if prop_val:
        labels_val = torch.tensor([float(classes.index(vdx)) for idx, vdx in enumerate(prop_val)]).long()

    ## dataset
    # train
    trainset = torch.utils.data.TensorDataset(z_train, labels_train)
    trainloader = FastTensorDataLoader(trainset, batch_size=10, shuffle=True)
    # test
    testset = torch.utils.data.TensorDataset(z_test, labels_test)
    testloader = FastTensorDataLoader(testset, batch_size=10, shuffle=True)
    # val
    if prop_val:
        valset = torch.utils.data.TensorDataset(z_val, labels_val)
        valloader = FastTensorDataLoader(valset, batch_size=10, shuffle=True)

    ## create model
    # configure model
    config = {}
    config["model::type"] = "MLP"
    config["model::h_dim"] = [100,100,100]
    # config["model::h_dim"] = []  # no hidden layers
    config["model::i_dim"] = z_train.shape[1]
    config["model::o_dim"] = no_classes
    config["model::init_type"] = "kaiming_normal"
    config["model::nlin"] = "tanh"
    config["model::dropout"] = 0
    config["model::use_bias"] = True
    config["optim::optimizer"] = "adam"
    config["optim::lr"] = 1e-4
    config["optim::wd"] = 1e-6
    config["training::task"] = "classification"
    config["training::batchsize"] = 5
    config["training::start_epoch"] = 1
    config["training::epochs_train"] = 15
    config["training::val_epochs"] = 10
    config["training::output_epoch"] = 10
    config["training::idx_out"] = 10
    config["training::checkpoint_dir"] = None
    config["training::tensorboard_dir"] = None
    config["seed"] = 42
    config["training::trainloader"] = trainloader
    config["training::testloader"] = testloader

    cuda = True if torch.cuda.is_available() else False
    MLP = NNmodule(config, cuda=cuda, verbosity=0)
    ## start training loop
    accs = {k : {'train': 0, 'test': 0, 'val': 0} for k in range(0, config["training::epochs_train"])}
    losses = {k : {'train': 0, 'test': 0, 'val': 0} for k in range(0, config["training::epochs_train"])}
    
    for idx in range(config["training::epochs_train"]):
        loss_train, acc_train = MLP.train_epoch(trainloader, epoch=-1)
        loss_test, acc_test = MLP.test_epoch(testloader, epoch=-1)
        if prop_val:
            loss_val, acc_val = MLP.test_epoch(valloader, epoch=-1)
        else:
            loss_val, acc_val = None, None
        accs[idx]['train'] = acc_train
        accs[idx]['test'] = acc_test
        accs[idx]['val'] = acc_val
        losses[idx]['train'] = loss_train
        losses[idx]['test'] = loss_test
        losses[idx]['val'] = loss_val
        if verbose: print(f'{idx} - train: {acc_train*100:3.3f} - val {acc_val*100:3.3f} - test {acc_test*100:3.3f}')
    return losses, accs

In [ ]:
def predict_asr(z_train, z_val, z_test, prop_train, prop_test, prop_val, verbose=True):
    ## get list of classes and class labels
    # cast to float and push to cpu. back to gpu will be done at dataloader time
    z_train = z_train.float().to(torch.device("cpu"))
    z_test = z_test.float().to(torch.device("cpu"))
    if z_val:
        z_val = z_val.float().to(torch.device("cpu"))
        
    # train set classes
    classes = list(np.unique(prop_train))
    no_classes = len(classes)
    # assert all classes are in trainset
    classes_test = list(np.unique(prop_test))
    assert set(classes_test).issubset(set(classes)), "test set contains classes which are not in train set"
    if prop_val:
        classes_val = list(np.unique(prop_val))
        assert set(classes_val).issubset(set(classes)), "val set contains classes which are not in train set"

    # one hot encoding
    labels_train = torch.tensor([float(classes.index(vdx)) for idx, vdx in enumerate(prop_train)]).long()
    labels_test = torch.tensor([float(classes.index(vdx)) for idx, vdx in enumerate(prop_test)]).long()
    if prop_val:
        labels_val = torch.tensor([float(classes.index(vdx)) for idx, vdx in enumerate(prop_val)]).long()

    ## dataset
    # train
    trainset = torch.utils.data.TensorDataset(z_train, labels_train)
    trainloader = FastTensorDataLoader(trainset, batch_size=10, shuffle=True)
    # test
    testset = torch.utils.data.TensorDataset(z_test, labels_test)
    testloader = FastTensorDataLoader(testset, batch_size=10, shuffle=True)
    # val
    if prop_val:
        valset = torch.utils.data.TensorDataset(z_val, labels_val)
        valloader = FastTensorDataLoader(valset, batch_size=10, shuffle=True)

    ## create model
    # configure model
    config = {}
    config["model::type"] = "MLP"
    config["model::h_dim"] = [100,100,100]
    # config["model::h_dim"] = []  # no hidden layers
    config["model::i_dim"] = z_train.shape[1]
    config["model::o_dim"] = no_classes
    config["model::init_type"] = "kaiming_normal"
    config["model::nlin"] = "tanh"
    config["model::dropout"] = 0
    config["model::use_bias"] = True
    config["optim::optimizer"] = "adam"
    config["optim::lr"] = 1e-4
    config["optim::wd"] = 1e-6
    config["training::task"] = "classification"
    config["training::batchsize"] = 5
    config["training::start_epoch"] = 1
    config["training::epochs_train"] = 15
    config["training::val_epochs"] = 10
    config["training::output_epoch"] = 10
    config["training::idx_out"] = 10
    config["training::checkpoint_dir"] = None
    config["training::tensorboard_dir"] = None
    config["seed"] = 42
    config["training::trainloader"] = trainloader
    config["training::testloader"] = testloader

    cuda = True if torch.cuda.is_available() else False
    MLP = NNmodule(config, cuda=cuda, verbosity=0)
    ## start training loop
    accs = {k : {'train': 0, 'test': 0, 'val': 0} for k in range(0, config["training::epochs_train"])}
    losses = {k : {'train': 0, 'test': 0, 'val': 0} for k in range(0, config["training::epochs_train"])}
    
    for idx in range(config["training::epochs_train"]):
        loss_train, acc_train = MLP.train_epoch(trainloader, epoch=-1)
        loss_test, acc_test = MLP.test_epoch(testloader, epoch=-1)
        if prop_val:
            loss_val, acc_val = MLP.test_epoch(valloader, epoch=-1)
        else:
            loss_val, acc_val = None, None
        accs[idx]['train'] = acc_train
        accs[idx]['test'] = acc_test
        accs[idx]['val'] = acc_val
        losses[idx]['train'] = loss_train
        losses[idx]['test'] = loss_test
        losses[idx]['val'] = loss_val
        if verbose: print(f'{idx} - train: {acc_train*100:3.3f} - val {acc_val*100:3.3f} - test {acc_test*100:3.3f}')
    return losses, accs

In [ ]:
# predict binary clean/trig
# embeddings
losses_embds, accs_embds = predict_poisoned(z_train=z_tr,z_val=z_vl,z_test=z_te,prop_train=p_tr,prop_test=p_te,prop_val=p_vl)
# raw weights
losses_wts, accs_wts = predict_poisoned(z_train=w_tr,z_val=w_vl,z_test=w_te,prop_train=p_tr,prop_test=p_te,prop_val=p_vl)